# Isolation Forest (IF) outlier detector deployment

Wrap a scikit-learn Isolation Forest python model for use as a prediction microservice in seldon-core and deploy on seldon-core running on minikube or a Kubernetes cluster using GCP.

## Dependencies

- [helm](https://github.com/helm/helm)
- [minikube](https://github.com/kubernetes/minikube) 
- [s2i](https://github.com/openshift/source-to-image) >= 1.1.13 

python packages:
- scikit-learn: pip install scikit-learn --> 0.20.1

## Task

The outlier detector needs to detect computer network intrusions using TCP dump data for a local-area network (LAN) simulating a typical U.S. Air Force LAN. A connection is a sequence of TCP packets starting and ending at some well defined times, between which data flows to and from a source IP address to a target IP address under some well defined protocol. Each connection is labeled as either normal, or as an attack. 

There are 4 types of attacks in the dataset:
- DOS: denial-of-service, e.g. syn flood;
- R2L: unauthorized access from a remote machine, e.g. guessing password;
- U2R:  unauthorized access to local superuser (root) privileges;
- probing: surveillance and other probing, e.g., port scanning.
    
The dataset contains about 5 million connection records.

There are 3 types of features:
- basic features of individual connections, e.g. duration of connection
- content features within a connection, e.g. number of failed log in attempts
- traffic features within a 2 second window, e.g. number of connections to the same host as the current connection

The outlier detector is only using 40 out of 41 features.

## Train locally

Train on small dataset where you roughly know the fraction of outliers, defined by the "contamination" parameter.

In [1]:
# define columns to keep
cols=['duration','protocol_type','flag','src_bytes','dst_bytes','land',
      'wrong_fragment','urgent','hot','num_failed_logins','logged_in',
      'num_compromised','root_shell','su_attempted','num_root','num_file_creations',
      'num_shells','num_access_files','num_outbound_cmds','is_host_login',
      'is_guest_login','count','srv_count','serror_rate','srv_serror_rate',
      'rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate',
      'srv_diff_host_rate','dst_host_count','dst_host_srv_count','dst_host_same_srv_rate',
      'dst_host_diff_srv_rate','dst_host_same_src_port_rate','dst_host_srv_diff_host_rate',
      'dst_host_serror_rate','dst_host_srv_serror_rate','dst_host_rerror_rate',
      'dst_host_srv_rerror_rate','target']
cols_str = str(cols)

In [2]:
!python train.py \
--dataset 'kddcup99' \
--samples 50000 \
--keep_cols "$cols_str" \
--contamination .1 \
--n_estimators 100 \
--max_samples .8 \
--max_features 1. \
--save_path './models/'


Load dataset

Generate training batch

Train outlier detector

Training done!


## Test using Kubernetes cluster on GCP or Minikube

Pick Kubernetes cluster on GCP or Minikube.

In [2]:
minikube = False

In [3]:
if minikube:
    !minikube start --memory 4096 --feature-gates=CustomResourceValidation=true \
    --extra-config=apiserver.Authorization.Mode=RBAC
else:
    !gcloud container clusters get-credentials standard-cluster-1 --zone europe-west1-b --project seldon-demos

Fetching cluster endpoint and auth data.
kubeconfig entry generated for standard-cluster-1.


Create a cluster-wide cluster-admin role assigned to a service account named “default” in the namespace “kube-system”.

In [4]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin \
--serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


In [5]:
!kubectl create namespace seldon

namespace/seldon created


Add current context details to the configuration file in the seldon namespace.

In [6]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "gke_seldon-demos_europe-west1-b_standard-cluster-1" modified.


Create tiller service account and give it a cluster-wide cluster-admin role.

In [7]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

serviceaccount/tiller created
clusterrolebinding.rbac.authorization.k8s.io/tiller created
$HELM_HOME has been configured at /home/arnaud/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.
Happy Helming!


Check deployment rollout status and deploy seldon/spartakus helm charts.

In [8]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

Waiting for deployment "tiller-deploy" rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out


In [9]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd \
    --set usage_metrics.enabled=true

NAME:   seldon-core-crd
LAST DEPLOYED: Tue Dec  4 14:53:17 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        0        0           0          1s

==> v1beta1/CustomResourceDefinition
NAME                                         KIND
seldondeployments.machinelearning.seldon.io  CustomResourceDefinition.v1beta1.apiextensions.k8s.io

==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        0s

==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     1s


NOTES:
NOTES: TODO




In [10]:
!helm install ../../../helm-charts/seldon-core --name seldon-core \
        --namespace seldon \
        --set ambassador.enabled=true

NAME:   seldon-core
LAST DEPLOYED: Tue Dec  4 14:53:19 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/ClusterRoleBinding
NAME           KIND                                             SUBJECTS
seldon-seldon  ClusterRoleBinding.v1.rbac.authorization.k8s.io  1 item(s)

==> v1beta1/Role
NAME          AGE
ambassador    1s
seldon-local  1s

==> v1beta1/RoleBinding
NAME        AGE
ambassador  1s

==> v1/RoleBinding
NAME    KIND                                      SUBJECTS
seldon  RoleBinding.v1.rbac.authorization.k8s.io  1 item(s)

==> v1/Service
NAME                          CLUSTER-IP    EXTERNAL-IP  PORT(S)                        AGE
seldon-core-ambassador-admin  10.7.243.27   <nodes>      8877:30877/TCP                 1s
seldon-core-ambassador        10.7.244.21   <nodes>      80:30645/TCP,443:32092/TCP     1s
seldon-core-seldon-apiserver  10.7.244.66   <nodes>      8080:32315/TCP,5000:32435/TCP  1s
seldon-core-redis             10.7.242.105  <none>       6379/TCP         

Check deployment rollout status for seldon core.

In [11]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager -n seldon
!kubectl rollout status deploy/seldon-core-seldon-apiserver -n seldon

Waiting for deployment "seldon-core-seldon-cluster-manager" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-seldon-cluster-manager" successfully rolled out
deployment "seldon-core-seldon-apiserver" successfully rolled out


If Minikube used: create docker image for outlier detector inside Minikube using s2i.

In [12]:
if minikube:
    !eval $(minikube docker-env) && s2i build . seldonio/seldon-core-s2i-python3:0.4-SNAPSHOT seldonio/outlier-if:0.1

Install outlier detector helm charts and set "threshold" hyperparameter value.

In [13]:
!helm install ../../../helm-charts/seldon-od-if \
    --set model.image.name=seldonio/outlier-if:0.1 \
    --set model.threshold=0.04 \
    --name outlier-detector --set oauth.key=oauth-key \
    --set oauth.secret=oauth-secret \
    --namespace=seldon

NAME:   outlier-detector
LAST DEPLOYED: Tue Dec  4 14:54:28 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1alpha2/SeldonDeployment
NAME              KIND
outlier-detector  SeldonDeployment.v1alpha2.machinelearning.seldon.io




## Port forward Ambassador

Run command in terminal:

```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

## Import rest requests, load data and test requests

In [14]:
from utils import get_payload, rest_request_ambassador, send_feedback_rest, get_kdd_data, generate_batch

data = get_kdd_data(keep_cols=cols) # load dataset
print(data.shape)

(4898431, 53)


Generate a random batch from the data

In [15]:
import numpy as np

samples = 1
fraction_outlier = 0.
X, labels = generate_batch(data,samples,fraction_outlier)
print(X.shape)
print(labels.shape)

(1, 52)
(1,)


Test the rest requests with the generated data. It is important that the order of requests is respected. First we make predictions, then we get the "true" labels back using the feedback request. If we do not respect the order and eg keep making predictions without getting the feedback for each prediction, there will be a mismatch between the predicted and "true" labels. This will result in errors in the produced metrics.

In [16]:
request = get_payload(X)

In [17]:
response = rest_request_ambassador("outlier-detector",request,endpoint="localhost:8003")

200
{
  "meta": {
    "puid": "1qlvfh1d89upve1707bd9co8d5",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
      "outlier-if": "seldonio/outlier-if:0.1"
    },
    "metrics": [{
      "key": "is_outlier",
      "type": "GAUGE",
      "value": "NaN",
      "tags": {
      }
    }, {
      "key": "anomaly_score",
      "type": "GAUGE",
      "value": "NaN",
      "tags": {
      }
    }, {
      "key": "observation",
      "type": "GAUGE",
      "value": 0.0,
      "tags": {
      }
    }, {
      "key": "threshold",
      "type": "GAUGE",
      "value": 0.04,
      "tags": {
      }
    }, {
      "key": "label",
      "type": "GAUGE",
      "value": "NaN",
      "tags": {
      }
    }, {
      "key": "accuracy_tot",
      "type": "GAUGE",
      "value": "NaN",
      "tags": {
      }
    }, {
      "key": "precision_tot",
      "type": "GAUGE",
      "value": "NaN",
      "tags": {
      }
    }, {
      "key": "recall_tot",
      "type": "GAUGE",
      "value": "Na

In [18]:
send_feedback_rest("outlier-detector",request,response,0,labels,endpoint="localhost:8003")

## Analytics

Install the helm charts for prometheus and the grafana dashboard

In [19]:
!helm install ../../../helm-charts/seldon-core-analytics --name seldon-core-analytics \
    --set grafana_prom_admin_password=password \
    --set persistence.enabled=false \
    --namespace seldon

NAME:   seldon-core-analytics
LAST DEPLOYED: Tue Dec  4 14:57:24 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/ServiceAccount
NAME        SECRETS  AGE
prometheus  1        2s

==> v1beta1/ClusterRole
NAME        AGE
prometheus  2s

==> v1/Job
NAME                            DESIRED  SUCCESSFUL  AGE
grafana-prom-import-dashboards  1        0           2s

==> v1beta1/Deployment
NAME                     DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
alertmanager-deployment  1        1        1           0          2s
grafana-prom-deployment  1        1        1           0          2s
prometheus-deployment    1        1        1           0          2s

==> v1beta1/DaemonSet
NAME                      DESIRED  CURRENT  READY  UP-TO-DATE  AVAILABLE  NODE-SELECTOR  AGE
prometheus-node-exporter  1        1        0      1           0          <none>         2s

==> v1/ConfigMap
NAME                       DATA  AGE
alertmanager-server-conf   1     2s
grafana-import-dashboards  9   

## Port forward Grafana dashboard

Run command in terminal:

```
kubectl port-forward $(kubectl get pods -n seldon -l app=grafana-prom-server -o jsonpath='{.items[0].metadata.name}') -n seldon 3000:3000
```

You can then view an analytics dashboard inside the cluster at http://localhost:3000/dashboard/db/prediction-analytics?refresh=5s&orgId=1. Your IP address may be different. get it via minikube ip. Login with:

Username : admin

password : password (as set when starting seldon-core-analytics above)

Import the outlier-detector-if dashboard from ../../../helm-charts/seldon-core-analytics/files/grafana/configs.

## Run simulation

- Sample random network intrusion data with a certain outlier probability.
- Get payload for the observation.
- Make a prediction.
- Send the "true" label with the feedback.

It is important that the prediction-feedback order is maintained. Otherwise there will be a mismatch between the predicted and "true" labels.

View the progress on the grafana "Outlier Detection" dashboard.

In [21]:
import time
n_requests = 100
samples = 1
for i in range(n_requests):
    fraction_outlier = .1
    X, labels = generate_batch(data,samples,fraction_outlier)
    request = get_payload(X)
    response = rest_request_ambassador("outlier-detector",request,endpoint="localhost:8003")
    send_feedback_rest("outlier-detector",request,response,0,labels,endpoint="localhost:8003")
    #time.sleep(1)